In [1]:
# Importing necessary libraries from TensorFlow and Keras
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, LeakyReLU, BatchNormalization, Reshape, Flatten, Input
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, Dropout
from tensorflow.keras.optimizers import Adam
import os

In [2]:
(X_train, _), (_, _) = mnist.load_data()
X_train = (X_train.astype(np.float32) - 127.5) / 127.5  # Normalize to [-1, 1]
X_train = np.expand_dims(X_train, axis=-1)  # Add channel dimension

In [3]:
# Improved Generator with added depth
def build_improved_generator(noise_dim=100):
    model = Sequential()

    # First layers
    model.add(Dense(256 * 7 * 7, activation='relu', input_dim=noise_dim))
    model.add(Reshape((7, 7, 256)))
    model.add(BatchNormalization(momentum=0.8))
    
    # Intermediate layer
    model.add(Conv2DTranspose(256, kernel_size=3, strides=1, padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    
    # Second layer
    model.add(Conv2DTranspose(128, kernel_size=3, strides=2, padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    
    # Third layer
    model.add(Conv2DTranspose(64, kernel_size=3, strides=2, padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    
    # Final layer
    model.add(Conv2D(1, kernel_size=3, padding='same', activation='tanh'))
    
    noise = Input(shape=(noise_dim,))
    img = model(noise)
    
    return Model(noise, img)

improved_generator = build_improved_generator()
improved_generator.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100)]             0         
                                                                 
 sequential (Sequential)     (None, 28, 28, 1)         2229249   
                                                                 
Total params: 2229249 (8.50 MB)
Trainable params: 2227841 (8.50 MB)
Non-trainable params: 1408 (5.50 KB)
_________________________________________________________________


In [4]:
# Improved Discriminator with Dropout layers for regularization
def build_improved_discriminator():
    model = Sequential()
    
    # First layer
    model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=(28, 28, 1), padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))  # Dropout layer added
    
    # Second layer
    model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))  # Dropout layer added
    
    # Third layer
    model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))  # Dropout layer added
    
    # Final layers
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid')) 
    
    img = Input(shape=(28, 28, 1))
    validity = model(img)
    
    return Model(img, validity)

improved_discriminator = build_improved_discriminator()
improved_discriminator.compile(loss='binary_crossentropy', optimizer=Adam(0.0002, 0.5), metrics=['accuracy'])
improved_discriminator.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 sequential_1 (Sequential)   (None, 1)                 94721     
                                                                 
Total params: 94721 (370.00 KB)
Trainable params: 94721 (370.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [5]:
# GAN Combinado using improved models
def build_improved_gan(generator, discriminator):
    discriminator.trainable = False
    z = Input(shape=(100,))
    img = generator(z)
    validity = discriminator(img)
    
    return Model(z, validity)

improved_gan = build_improved_gan(improved_generator, improved_discriminator)
improved_gan.compile(loss='binary_crossentropy', optimizer=Adam(0.0002, 0.5))
improved_gan.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 100)]             0         
                                                                 
 model (Functional)          (None, 28, 28, 1)         2229249   
                                                                 
 model_1 (Functional)        (None, 1)                 94721     
                                                                 
Total params: 2323970 (8.87 MB)
Trainable params: 2227841 (8.50 MB)
Non-trainable params: 96129 (375.50 KB)
_________________________________________________________________


In [6]:
# Directory for saving generated images
if not os.path.exists('generated_images'):
    os.mkdir('generated_images')

In [7]:
# Training function
def train_gan(generator, discriminator, gan, epochs, batch_size=128, save_interval=500):
    # Labels for real and fake data
    valid = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))

    for epoch in range(epochs):

        # Train Discriminator

        # Select random batch of real images
        idx = np.random.randint(0, X_train.shape[0], batch_size)
        real_imgs = X_train[idx]

        # Generate batch of fake images
        noise = np.random.normal(0, 1, (batch_size, 100))
        fake_imgs = generator.predict(noise)

        # Train discriminator
        d_loss_real = discriminator.train_on_batch(real_imgs, valid)
        d_loss_fake = discriminator.train_on_batch(fake_imgs, fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # Train Generator

        # Generate batch of noise vectors
        noise = np.random.normal(0, 1, (batch_size, 100))

        # Train generator to make discriminator classify generated images as valid
        g_loss = gan.train_on_batch(noise, valid)

       # If at save interval, save generated image samples and print loss
        if epoch % save_interval == 0:
            print(f"{epoch}/{epochs} [D loss: {d_loss[0]} | D accuracy: {100 * d_loss[1]}] [G loss: {g_loss}]")
            save_imgs(generator, epoch)  # Pasa el generador como argumento

        # Optionally, save model weights at intervals
        if epoch % (save_interval * 10) == 0:
            generator.save(f'generator_{epoch}.h5')
            discriminator.save(f'discriminator_{epoch}.h5')

In [8]:
# Function to save generated images
def save_imgs(generator, epoch):
    noise = np.random.normal(0, 1, (10, 100))
    generated_imgs = generator.predict(noise)
    generated_imgs = 0.5 * generated_imgs + 0.5  # Rescale to [0, 1]
    
    fig, axs = plt.subplots(1, 10)
    for i in range(10):
        axs[i].imshow(generated_imgs[i, :, :, 0], cmap='gray')
        axs[i].axis('off')
    plt.savefig(f"generated_images/image_at_epoch_{epoch}.png")
    plt.close()

In [9]:
def evaluate_discriminator(model, real_samples, fake_samples):
    # Clasificar las imágenes reales
    _, acc_real = model.evaluate(real_samples, np.ones((len(real_samples), 1)), verbose=0)
    
    # Clasificar las imágenes falsas
    _, acc_fake = model.evaluate(fake_samples, np.zeros((len(fake_samples), 1)), verbose=0)
    
    # Calcular la precisión del discriminador
    acc = (acc_real + acc_fake) / 2
    return acc_real, acc_fake, acc

In [10]:
# Training the GAN with adjusted parameters for hardware
train_gan(improved_generator, improved_discriminator, improved_gan, epochs=30000, batch_size=32, save_interval=500)

1/1 [==============================] - 0s 233ms/step
0/30000 [D loss: 0.6785685420036316 | D accuracy: 37.5] [G loss: 0.6716071367263794]
1/1 [==============================] - 0s 29ms/step


C:\Users\Gamer\AppData\Roaming\Python\Python310\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1/1 [==============================] - 0s 30ms/step


1/1 [==============================] - 0s 32ms/step


1/1 [==============================] - 0s 33ms/step


...

1/1 [==============================] - 0s 141ms/step


1/1 [==============================] - 0s 142ms/step
29500/30000 [D loss: 0.6992250382900238 | D accuracy: 50.0] [G loss: 0.7069070339202881]
1/1 [==============================] - 0s 142ms/step


1/1 [==============================] - 0s 147ms/step


1/1 [==============================] - 0s 146ms/step


1/1 [==============================] - 0s 145ms/step


In [11]:
# Genera algunas imágenes con tu generador
num_samples = 1000
noise = np.random.normal(0, 1, (num_samples, 100))
generated_images = improved_generator.predict(noise)

# Selecciona un número similar de imágenes reales
real_samples = X_train[np.random.randint(0, X_train.shape[0], num_samples)]

# Evaluar el discriminador
acc_real, acc_fake, acc = evaluate_discriminator(improved_discriminator, real_samples, generated_images)
print(f"Accuracy real samples: {acc_real * 100:.2f}%")
print(f"Accuracy fake samples: {acc_fake * 100:.2f}%")
print(f"Overall accuracy: {acc * 100:.2f}%")

32/32 [==============================] - 4s 129ms/step
Accuracy real samples: 86.40%
Accuracy fake samples: 29.10%
Overall accuracy: 57.75%
